In [48]:
import datetime

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession

In [ ]:
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [2]:
# import pandas as pd
# import datetime

# # import data
# data = pd.read_csv("/home/bluepi/Downloads/Update/product_info/main table/product_info.csv")

# # convert day column type to datetime
# data['day'] = pd.to_datetime( data['day'], infer_datetime_format=True, yearfirst=True)

# # create new column
# data['date_timestamp'] = pd.to_datetime(data.day.astype(str) + ' ' + data.time)

# # Drop old columns
# data.drop(['day','time'],inplace=True,axis=1)

# # Write to csv
# data.to_csv("/home/bluepi/Downloads/Update/product_info/main table/main table.csv", index=False)

In [3]:
mainTable = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main table.csv")

In [4]:
mainTable.printSchema()

root
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- date_timestamp: timestamp (nullable = true)



In [36]:
from pyspark.sql.functions import year, month, dayofyear, hour, minute, second

mainTable.select([year(mainTable['date_timestamp']).alias("Year"),
                  month(mainTable['date_timestamp']).alias("Month"),
                  dayofyear(mainTable['date_timestamp']).alias("DayOfYear"),
                  hour(mainTable['date_timestamp']).alias("Hour"),
                  minute(mainTable['date_timestamp']).alias("Minute"),
                  second(mainTable['date_timestamp']).alias("Second")]).show(3)

+----+-----+---------+----+------+------+
|Year|Month|DayOfYear|Hour|Minute|Second|
+----+-----+---------+----+------+------+
|2020|    9|      247|  19|    24|    25|
|2020|    3|       78|  16|    41|    43|
|2020|    8|      216|  22|    17|     5|
+----+-----+---------+----+------+------+
only showing top 3 rows

None


In [47]:
mainTable.select('date_timestamp').orderBy(mainTable.date_timestamp.asc()).show(20)

+-------------------+
|     date_timestamp|
+-------------------+
|2020-01-03 04:07:54|
|2020-01-03 04:38:21|
|2020-01-03 04:41:13|
|2020-01-03 05:24:56|
|2020-01-03 22:27:04|
|2020-01-03 23:54:49|
|2020-02-03 04:05:43|
|2020-02-03 04:35:29|
|2020-02-03 05:02:57|
|2020-02-03 06:12:33|
|2020-02-03 10:30:17|
|2020-02-03 10:58:29|
|2020-02-03 14:36:42|
|2020-02-03 16:38:45|
|2020-02-03 17:03:12|
|2020-02-03 18:43:16|
|2020-02-03 23:43:02|
|2020-03-03 03:01:52|
|2020-03-03 04:20:22|
|2020-03-03 04:30:40|
+-------------------+
only showing top 20 rows



In [64]:
# Address to the product_info folder
address = "/home/bluepi/Downloads/Update/product_info/"
previous_day = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%d-%m-%Y')

# Address to the previous day folder
new_address = address + previous_day

# # read the previous day folder
data = spark.read.format('csv') \
          .options( header=True, inferschema=True ) \
          .load(new_address)

In [65]:
data.show(3)

+----+----------+-----+-------------------+-----------+
|p_id|    p_name|price|     Date_timestamp|record_type|
+----+----------+-----+-------------------+-----------+
| 862|       Tin|  457|2020-03-24 11:15:59|          U|
| 184|Voltsillam|  671|2020-03-24 12:20:00|          U|
| 642|   Cardify|  885|2020-03-24 11:41:13|          U|
+----+----------+-----+-------------------+-----------+
only showing top 3 rows



In [66]:
data.orderBy(data.Date_timestamp.asc()).show()

+----+---------+-----+-------------------+-----------+
|p_id|   p_name|price|     Date_timestamp|record_type|
+----+---------+-----+-------------------+-----------+
| 127|  Andalax|  479|2020-03-24 04:36:35|          I|
| 980|   Lotlux|  618|2020-03-24 04:37:04|          I|
| 150|  Flexidy|  189|2020-03-24 04:46:52|          U|
| 856|  Konklab|  677|2020-03-24 04:57:44|          I|
| 910|    Alpha|  384|2020-03-24 05:05:08|          U|
| 459|  Regrant|  454|2020-03-24 05:05:12|          U|
| 625|Gembucket|  192|2020-03-24 05:12:03|          U|
| 154|  Redhold|   62|2020-03-24 05:14:12|          I|
| 260|     Stim|  998|2020-03-24 05:16:04|          U|
| 764|  Fixflex|  446|2020-03-24 05:20:20|          I|
| 163|    Asoka| 1387|2020-03-24 05:22:02|          U|
| 764|  Fixflex|  446|2020-03-24 11:00:50|          I|
| 625|Gembucket|  588|2020-03-24 11:12:12|          U|
| 888|  Fix San| 1158|2020-03-24 11:13:53|          U|
| 862|      Tin|  457|2020-03-24 11:15:59|          U|
| 298|   T